In [9]:
import pandas as pd
import os, glob

from owlready2 import *
import owlready2
print(owlready2.VERSION)


import importlib.util
import sys
spec = importlib.util.spec_from_file_location("rdfutils", "../../../utils/rdfutils.py")
u = importlib.util.module_from_spec(spec)
sys.modules["rdfutils"] = u
spec.loader.exec_module(u)

from datetime import datetime

def NOW():
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    return "Current Time = "+ str(current_time)

%load_ext autoreload
%autoreload 2


0.40
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
onto = get_ontology("./WIP.owl").load()

In [6]:
TECHS = [x.label[0] for x in onto.Technology.instances() if len(x.label) >= 1]
len(TECHS)

23090

# Classification

In [4]:
import pandas as pd
import json
import glob, os
import requests
from datetime import datetime
import hashlib

%load_ext autoreload
%autoreload 2

from dotenv import load_dotenv
load_dotenv('.env')

URL = os.getenv("KG_URL_FCT")
TOK3N = os.getenv("KG_TOKEN")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
CONTEXT = """You are given a list of technologies. Classify them using 10 groups of technologies you define. Answer only as a bullet point list structure, similar to:
## Name of technologies group
- technology A
- technology C 
## Name of technologies group
- technology B
etc..
"""
NChunks = 1000

NB = len(TECHS)//NChunks
CATS = []
for k in range(NB):
    QUESTION =  "The technologies are:\n\n* "+"\n* ".join(TECHS[(k*NChunks):(k+1)*NChunks]) 
    REQ = {
        "context": CONTEXT,
        "question": QUESTION,
        "token": TOK3N,
        "overwrite": True,
        "source": "local-evaltechsT3.5"
    }

    x = requests.post(URL+"ask/", json = REQ)
    CATS.append(json.loads(x.text)["answer"])

In [14]:
titles = []
for k in range(len(CATS)):
    [titles.append(x.replace("#","").strip()) for x in CATS[k].split("\n") if x.startswith("#")]
    titles.sort()
pd.DataFrame(titles).to_excel("../../support/classif_auto_techs.xlsx")

In [41]:
techsList = ["Building Management", "Sensors and Monitoring", "New technologies","Financial Technologies", 'Healthcare', "Supply-chain", "Air Control","Building design","Infrastructure Management", "Water Control", "Communication", "Data Analytics", "Privacy Enhancing Technologies",  "Information Management", "Cleaning", "Research and development", "Materials", "Urban Planning" ]
print(len(techsList))
techsList.sort()
techsList

18


['Air Control',
 'Building Management',
 'Building design',
 'Cleaning',
 'Communication',
 'Data Analytics',
 'Financial Technologies',
 'Healthcare',
 'Information Management',
 'Infrastructure Management',
 'Materials',
 'New technologies',
 'Privacy Enhancing Technologies',
 'Research and development',
 'Sensors and Monitoring',
 'Supply-chain',
 'Urban Planning',
 'Water Control']

In [42]:
classifFct = [
        {
        "name": "classify_risks",
        "description": "Function used to classify risks.",
        "parameters": {
            "type": "object",
            "properties": {
                "sorted_risks": {
                    "type": 'array',
                    "items": {
                        "type": 'object',
                        "description": "A type of risk ",
                        "properties": {
                            "name" :{
                                "type": 'string', 
                                "description": 'The risk being considered.'
                            },
                            "category" :{
                                "type": 'string', 
                                "enum":techsList,
                                "description": 'The category of risk that matches the risk category being considered.'
                            },
                            "confidence" :{
                                "type": 'string',  
                                "enum":["High","Medium-high","Medium","Low"],
                                "description": 'Confidence that this is the correct risk category.'
                            }                            
                        },
                        "required": ["name",'category',"confidence"],
                    }
                },
            },
            "required": ["sorted_risks"],
        },
    }
]

In [30]:
import importlib.util
import sys
spec = importlib.util.spec_from_file_location("llm", "../../../utils/llm.py")
h = importlib.util.module_from_spec(spec)
sys.modules["llm"] = h
spec.loader.exec_module(h)

In [36]:
RES = []
NChunks = 50

NB = len(TECHS)//NChunks
CATS = []
for k in range(NB):
    try:
        QUESTION =  "The technologies are:\n\n* "+"\n* ".join(TECHS[(k*NChunks):(k+1)*NChunks]) 

        #answer = ask("Out the following list, classify the risks  \n\n",Rs,prefix="risk_classification",functions=classifFct,overwrite=True)
        prefix ="ff"
        overwrite = False
        REQ = {
            "context": "Out the following list, classify the technologies\n\n",
            "question": QUESTION,
            "functions": classifFct,
            "token": TOK3N,
            "overwrite": overwrite,
            "source": "local-evalrisksT3.5"
        }

        H = h.hashme(REQ["context"]+REQ["question"])
        cached = "cache/"+prefix+"_"+H+".json"
        if not os.path.isfile(cached) or overwrite:
            x = requests.post(URL+"fct/", json = REQ)
            answer = json.loads(x.text)["messages"][-1]
            h.svt(cached,answer)
        else:
            answer = h.ldt(cached)
        CATS.append(pd.DataFrame(json.loads(answer["function_call"]["arguments"])["sorted_risks"]))
    except:
        pass

In [61]:
df = pd.concat(CATS)[["name","category","confidence"]]
df.to_parquet("newtechs_classified.parquet.gzip",compression="gzip")
df

,name,category,confidence
0,Nan,New technologies,Medium-high
1,Floor markings,Building design,Medium
2,Hvac systems,Air Control,High
3,Signage,Communication,Medium-high
4,Physical barriers,Building Management,High
...,...,...,...
46,Financial risk assessment models,New technologiesFinancial Technologies,High
47,Advanced healthcare management technologies,Healthcare,High
48,Advanced infection detection technologies,Healthcare,High
49,Healthcare management technologies,Healthcare,High


In [62]:
df[df.category == "Air Control"]

,name,category,confidence
2,Hvac systems,Air Control,High
6,Air purifiers,Air Control,High
7,Hvac technology,Air Control,High
34,Ventilation systems,Air Control,High
36,Lighting and hvac systems,Air Control,High
...,...,...,...
7,Air quality control in transportation and cons...,Air Control,High
16,Non-recirculating air ventilation and conditio...,Air Control,High
14,Smart air filtration systems,Air Control,High
33,Ventilation and air control,Air Control,High


In [119]:
df.groupby("category").name.count().reset_index().sort_values(by="name",ascending=False)[20:30]

,category,name
62,Education,72
207,Technology,50
185,Security,42
231,Waste Management,20
126,Marketing,20
78,Environmental Management,18
33,Community Engagement,16
230,Waste Control,15
142,Policy,15
67,Emergency Planning,13


In [121]:
NAMES = [str(x) for x in list(df[df.category == "Waste Management"].name.unique())]
QUESTION =  "The technologies are:\n\n* "+"\n* ".join(NAMES[:1000]) 

#answer = ask("Out the following list, classify the risks  \n\n",Rs,prefix="risk_classification",functions=classifFct,overwrite=True)
prefix ="tech_hvac"
overwrite = True
REQ = {
    "context": "Give  me an unnumbered bullet point list of 10 technologies groups, which could capture the following list of technologies. Don't use 'technologies' in their names.",
    "question": QUESTION,
    "token": TOK3N,
    "overwrite": overwrite,
    "source": "local-findsthT3.5"
}

H = h.hashme(REQ["context"]+REQ["question"])
cached = "cache/"+prefix+"_"+H+".json"
if not os.path.isfile(cached) or overwrite:
    x = requests.post(URL+"ask/", json = REQ)
    answer = json.loads(x.text)["answer"]
    h.svt(cached,answer)
else:
    answer = h.ldt(cached)

In [122]:
print([x.strip() for x in answer[2:].split("\n- ")])

['Recycling and waste handling technologies', 'Waste management and disposal solutions', 'Advanced waste sorting and processing systems', 'Smart waste handling and monitoring technologies', 'Contactless recycling and waste disposal solutions', 'Automated waste collection and segregation technologies', 'Remote waste management and monitoring systems', 'Sanitization and sterilization technologies for waste', 'Innovative waste separation and recycling methods', 'Improved waste storage and disposal systems']


In [123]:
list(set(['HVAC', 'Air Purification', 'Ventilation', 'Air Filtering', 'UV-C Disinfection', 'Monitoring Systems']+['Air Purification', 'HVAC Systems', 'Ventilation', 'Lighting', 'Indoor Environment', 'Air Filters', 'Portable Air Purifiers', 'HEPA Filters', 'Ultraviolet (UV) Technology', 'Air Quality Monitoring']))

['Air Filtering',
 'Indoor Environment',
 'HVAC Systems',
 'Air Quality Monitoring',
 'Monitoring Systems',
 'Air Purification',
 'HVAC',
 'Lighting',
 'Ultraviolet (UV) Technology',
 'Ventilation',
 'Air Filters',
 'Portable Air Purifiers',
 'HEPA Filters',
 'UV-C Disinfection']

In [124]:
DICT = {}
DICT["Air Control"] = ['Air Filtering',
 'Indoor Environment',
 'HVAC Systems',
 'Air Quality Monitoring',
 'Air Purification',
 'Ventilation',
 'HEPA Filters',
 'UV-C Disinfection']
DICT["Data Analytics"] = ['Machine Learning and AI Algorithms', 'Real-Time Data Tracking and Monitoring', 'Data Visualization and Modeling', 'Cloud-Based Data Sharing and Analysis', 'Advanced Statistical Modeling and Analysis', 'Social Media Monitoring and Analysis', 'Predictive Analytics and Modeling Tools',  'Information Dissemination Platforms']
DICT["Infrastructure Management"] = ['Sanitation and hygiene facilities', 'Ventilation and air quality systems', 'Geographic information and mapping technology', 'Healthcare infrastructure and telecommunication networks', 'Energy-efficient technologies and smart grid systems', 'Remote work tools and workforce management technologies', 'Logistics and inventory management systems', 'Capacity-building and infrastructure development tools', 'Disaster management and resilience planning technologies']
DICT["Communication"] = ['Communication and Collaboration Platforms', 'Public Awareness Campaign Tools', 'Video Conferencing and Collaboration Software', 'Social Media Platforms', 'Virtual Communication and Collaboration Technologies', 'Digital Signage and Messaging Systems', 'Multilingual Communication and Outreach Tools', 'Online Public Engagement Platforms', 'Remote Work and Telecommuting Technologies', 'Mobile App Notification Systems']
DICT["Sensors and Monitoring"] = ['Monitoring solutions', 'IoT devices', 'Surveillance systems', 'Environmental sensors', 'Sensor technologies', 'Remote monitoring tools', 'Crowd monitoring technology', 'Data collection systems', 'Real-time tracking systems']
DICT["Healthcare"] = ['Diagnostic Testing', 'Sequencing and Genomic Analysis', 'Analytical Chemistry', 'Physical Health Monitoring', 'Wellbeing and Mental Health', 'Social Distancing Technologies', 'Vaccination Development and Distribution', 'Personal Hygiene and Sanitation', 'Personal Protective Equipment (PPE)', 'Remote Healthcare and Telemedicine']
DICT["Urban Planning"] = ['Community Management Technologies', 'Spatial Planning and Layout Tools', 'Green Infrastructure Management', 'Crowd Control Solutions', 'Smart City Technologies', 'Urban Planning Solutions', 'Resilience Planning Frameworks', 'Urban Design and Mapping Tools', 'Environmental Conservation Technologies']
DICT["Digital and Information Management"] = ['Data Management', 'Web Development', 'Network Security', 'Cloud Services', 'Mobile Applications', 'Knowledge Management', 'Ontology Tools', 'Data Governance', 'Digital Infrastructure', 'Information Systems']
DICT["Building design"] = ['Building design', 'Ventilation systems', 'Home design and architecture', 'Sustainable design', 'Soundproofing technology', 'Green building strategies', 'Space optimization technologies', 'Architectural design software', 'Adaptive construction methods', 'Virtual planning and design tools']
DICT["Privacy Enhancing Technologies"] = ['Encryption and authentication', 'Biometrics and identification', 'Face and voice recognition', 'Data encryption and security', 'Cybersecurity and access control', 'Quantum cryptography', 'Ethical guidelines and advocacy', 'Fact-checking and information verification', 'Blockchain for data privacy and transparency']
DICT["Building Management"] = ['Access control and security technology', 'Lighting and daylighting systems', 'Maintenance and landscaping equipment', 'Door-locking and automatic systems', 'Operation of HVAC and ventilation systems', 'Building management and automation systems', 'Industry standard compliance tools', 'Safety and hazard detection systems']
DICT["Cleaning"] = ['Automated sanitation equipment', 'Advanced disinfection systems', 'Surface cleaning', 'Waste management technologies group', 'Sanitization equipment group', 'Disinfectant formulation', 'Self-cleaning technologies', 'Hygiene maintenance tools']
DICT["Supply-chain"] = ['Diversification and suppliers', 'Supply chain and risk management',  'Manufacturing of PPE', 'Digital supply chain management', 'Contactless delivery']
DICT["Water Control"] = ['Water and Wastewater Management', 'Water Treatment Systems', 'Water Quality Monitoring', 'Water Disinfection Solutions',  'Advanced Filtration Technologies', 'Sewage Treatment Facilities', 'Water Recycling and Conservation']
DICT["Waste Management"] = ['Recycling and waste handling technologies', 'Advanced waste sorting and processing systems', 'Smart waste handling and monitoring technologies', 'Contactless recycling and waste disposal solutions', 'Automated waste collection and segregation technologies', 'Sanitization and sterilization technologies for waste', 'Improved waste storage and disposal systems']



In [125]:
ans = {}
for stuff in DICT.keys():
    ans[stuff] = []
    classifFct = [
            {
            "name": "classify_technology",
            "description": "Function used to classify technology.",
            "parameters": {
                "type": "object",
                "properties": {
                    "sorted_tech": {
                        "type": 'array',
                        "items": {
                            "type": 'object',
                            "description": "A type of technology ",
                            "properties": {
                                "name" :{
                                    "type": 'string', 
                                    "description": 'The technology being considered.'
                                },
                                "category" :{
                                    "type": 'string', 
                                    "enum":DICT[stuff],
                                    "description": 'The category of technology that matches the technology category being considered.'
                                },
                                "confidence" :{
                                    "type": 'string',  
                                    "enum":["High","Medium-high","Medium","Low"],
                                    "description": 'Confidence that this is the correct technology category.'
                                }                            
                            },
                            "required": ["name",'category',"confidence"],
                        }
                    },
                },
                "required": ["sorted_tech"],
            },
        }
    ]


    NChunks = 50
    NAME = [str(x) for x in list(df[df.category == stuff].name.unique())]
    NB = len(NAME)//NChunks
    for k in range(NB):
        try:
            QUESTION =  "The technologies are:\n\n* "+"\n* ".join(NAME[(k*NChunks):(k+1)*NChunks]) 

            #answer = ask("Out the following list, classify the risks  \n\n",Rs,prefix="risk_classification",functions=classifFct,overwrite=True)
            prefix ="ff"
            overwrite = False
            REQ = {
                "context": "Out the following list, classify the technologies\n\n",
                "question": QUESTION,
                "functions": classifFct,
                "token": TOK3N,
                "overwrite": overwrite,
                "source": "local-evalrisksT3.5"
            }

            H = h.hashme(REQ["context"]+REQ["question"])
            cached = "cache/"+prefix+"_"+H+".json"
            if not os.path.isfile(cached) or overwrite:
                x = requests.post(URL+"fct/", json = REQ)
                answer = json.loads(x.text)["messages"][-1]
                h.svt(cached,answer)
            else:
                answer = h.ldt(cached)
            ans[stuff].append(pd.DataFrame(json.loads(answer["function_call"]["arguments"])["sorted_tech"]))
        except:
            pass

TypeError: 'builtin_function_or_method' object is not iterable